In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import pickle

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [2]:
cluster = 4

In [3]:
data = pd.read_csv(f'D:\\庫存健診開發\\data\\Training\\Raw\\Cluster_{cluster}.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,investment_buy_kurtosis,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3
0,2007-07-02,1305,華夏,5.405789,5.527295,5.304379,5.404321,15024.854877,140751.912437,37.0,...,NaN,NaN,NaN,NaN,NaN,0.222916,0.222916,0.000000,0.000000,0.000000
1,2007-07-03,1305,華夏,5.439230,5.560106,5.333371,5.431738,7842.164756,81469.893154,36.3,...,NaN,NaN,NaN,39.784904,NaN,0.226734,0.224953,0.000000,0.000000,0.000000
2,2007-07-04,1305,華夏,5.473129,5.593361,5.362762,5.459538,11817.484707,116903.896133,36.8,...,NaN,NaN,NaN,60.481869,100.266774,0.230599,0.227091,0.000000,0.000000,0.000000
3,2007-07-05,1305,華夏,5.507360,5.626936,5.392442,5.487619,17157.233337,164107.493915,38.5,...,NaN,NaN,NaN,88.246509,148.728378,0.234495,0.229327,0.000000,0.000000,0.000000
4,2007-07-06,1305,華夏,5.543516,5.662407,5.423792,5.517278,27296.036060,265434.394565,41.2,...,NaN,NaN,NaN,147.530742,235.777251,0.238615,0.231711,0.000000,0.594645,0.306914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308639,2019-09-17,9958,世紀鋼,76.599123,77.473952,75.712311,76.193901,4318.768378,339531.271421,161.8,...,2.011113,0.659924,-0.142467,-4.240093,6.017040,1.761640,2.045825,0.830230,0.424065,0.506313
308640,2019-09-18,9958,世紀鋼,76.539468,77.265808,75.649237,75.968260,3068.071378,235454.760807,162.6,...,2.856024,0.577861,-0.270728,-9.085778,-13.325871,1.616571,1.992168,0.811675,0.321372,0.413843
308641,2019-09-19,9958,世紀鋼,76.571516,77.196330,75.687225,75.893182,2462.163409,185428.207154,159.7,...,2.852656,0.670798,-0.297717,-2.433332,-11.519110,1.509105,1.931785,0.811675,0.243049,0.328446
308642,2019-09-20,9958,世紀鋼,76.671053,77.246500,75.801507,75.950037,2299.101425,176377.424777,158.6,...,6.861275,0.612239,-0.238165,1.722372,-0.710960,1.444993,1.870936,0.815948,0.160086,0.244266


In [4]:
train_df, val_df, test_df = func.TrainTestSplit(data, date(2016,1,1), date(2017,9,1))
del data

D:\庫存健診開發\code\TrainingPrep\Training_function.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_df = data[data['ts'].dt.date < TrainDate][data['ema26'].notnull()]
D:\庫存健診開發\code\TrainingPrep\Training_function.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_df = data[data['ts'].dt.date < valDate][data['ts'].dt.date >= TrainDate][data['ema26'].notnull()]
D:\庫存健診開發\code\TrainingPrep\Training_function.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = data[data['ts'].dt.date >= valDate][data['ema26'].notnull()]


In [5]:
feature_list = ['open', 'high', 'low', 'close', 'vol',
       'VWAP', 'index_open', 'index_high', 'index_low', 'index_close', 'index_vol',
       'industry_open', 'industry_high', 'industry_low', 'industry_close',
       'industry_vol', 'foreign_buy', 'investment_buy', 'dealer_buy',
       'ma7', 'ma21', 'ema26', 'ema12', 'MACD', 'upper_band',
       'lower_band', 'index_close_corr', 'industry_close_corr',
       'foreign_buy_corr', 'investment_buy_corr', 'dealer_buy_corr',
       'close_skew', 'close_kurtosis', 'index_close_skew',
       'index_close_kurtosis', 'industry_close_skew',
       'industry_close_kurtosis', 'foreign_buy_skew', 'foreign_buy_kurtosis',
       'investment_buy_skew', 'investment_buy_kurtosis', 'dealer_buy_skew',
       'dealer_buy_kurtosis','pvt', 'ATR', 'RSI_15', 'SO5', 'SO3']

print(len(feature_list))
print(train_df[feature_list].isna().values.any())
print(val_df[feature_list].isna().values.any())
print(test_df[feature_list].isna().values.any())

48
False
False
False


In [6]:
param = func.GetScalerParam(train_df, feature_list)
param

,max,min,std,mean
open,5.829502e+02,1.841155e+00,5.910401e+01,4.564717e+01
high,5.930570e+02,1.886864e+00,6.004004e+01,4.636774e+01
low,5.794671e+02,1.791730e+00,5.808834e+01,4.487714e+01
close,5.845570e+02,1.837296e+00,5.896711e+01,4.553538e+01
vol,3.041055e+05,-6.699386e+04,9.381440e+03,5.712207e+03
VWAP,9.410476e+02,0.000000e+00,7.453722e+01,5.921009e+01
index_open,9.958812e+03,4.113681e+03,1.118504e+03,8.011886e+03
index_high,1.000750e+04,4.184313e+03,1.113376e+03,8.049764e+03
index_low,9.905752e+03,4.060512e+03,1.121332e+03,7.959118e+03
index_close,9.954826e+03,4.131616e+03,1.115894e+03,8.002579e+03


In [7]:
train_df, val_df, test_df = func.Scaler(train_df, feature_list, param), func.Scaler(val_df, feature_list, param), func.Scaler(test_df, feature_list, param)
display(train_df)
display(test_df)

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,investment_buy_kurtosis,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3
24,2007-08-03,1305,華夏,0.006196,0.006191,0.006097,0.006115,0.205422,93473.562975,37.8,...,0.000000,0.648448,0.268610,-37.875978,0.229528,0.233237,0.006971,0.130903,0.324913,0.284617
25,2007-08-06,1305,華夏,0.006158,0.006153,0.006061,0.006079,0.204153,88767.025328,38.0,...,0.000000,0.648511,0.268634,-33.712276,0.229698,0.231245,0.006918,0.112707,0.331635,0.288982
26,2007-08-07,1305,華夏,0.006117,0.006111,0.006022,0.006041,0.203977,88488.090397,38.1,...,0.000000,0.651649,0.271862,-35.943166,0.229850,0.229085,0.006862,0.096309,0.335872,0.293713
27,2007-08-08,1305,華夏,0.006073,0.006066,0.005980,0.006000,0.203558,86965.095578,38.9,...,0.000000,0.647103,0.268616,-38.248010,0.229492,0.226718,0.006803,0.081636,0.336442,0.296421
28,2007-08-09,1305,華夏,0.006032,0.006024,0.005941,0.005961,0.204914,91502.037141,39.5,...,0.000000,0.641081,0.268811,-38.481561,0.229292,0.224541,0.006741,0.070122,0.337956,0.298686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244245,2015-12-25,9938,百和,0.108562,0.107756,0.107154,0.107599,0.184294,92647.922011,222.3,...,0.375758,0.598939,0.267158,1.770895,0.236201,1.896895,0.077464,0.555489,0.638466,0.727867
244246,2015-12-28,9938,百和,0.109128,0.108014,0.107290,0.107725,0.185057,111110.052284,219.0,...,0.384488,0.602970,0.262893,1.906329,0.235638,1.971022,0.076285,0.579361,0.540167,0.636018
244247,2015-12-29,9938,百和,0.109266,0.108184,0.107672,0.107896,0.185031,110950.917869,215.2,...,0.380269,0.599307,0.261660,2.579307,0.235702,1.850913,0.074723,0.595699,0.536283,0.587757
244248,2015-12-30,9938,百和,0.109056,0.108103,0.107682,0.107814,0.186315,147223.522653,216.9,...,0.415373,0.603833,0.262989,-1.583712,0.235427,1.797110,0.073118,0.595699,0.482840,0.531487


,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,investment_buy_kurtosis,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3
244656,2017-09-01,1305,華夏,0.047018,0.046868,0.046692,0.046908,0.194616,181481.573240,172.2,...,0.381091,0.620096,0.216366,98.502580,0.243641,0.961727,0.033040,0.999988,0.707148,0.672046
244657,2017-09-04,1305,華夏,0.047087,0.046868,0.046733,0.046916,0.190285,127626.097381,172.4,...,0.381512,0.629247,0.188708,0.533230,0.243165,0.805632,0.032266,0.999988,0.700175,0.704336
244658,2017-09-05,1305,華夏,0.047162,0.046866,0.046776,0.046919,0.188164,100500.829678,170.7,...,0.381790,0.623388,0.187681,0.212019,0.235407,0.779710,0.031475,0.999988,0.689592,0.714116
244659,2017-09-06,1305,華夏,0.047240,0.046863,0.046821,0.046919,0.186299,74979.875063,170.7,...,0.387111,0.621168,0.188896,-0.014576,0.235364,0.751660,0.030659,0.999988,0.673740,0.709473
244660,2017-09-07,1305,華夏,0.047342,0.046862,0.046881,0.046923,0.199154,247609.733049,177.1,...,0.386747,0.593292,0.163083,0.594413,0.235394,0.716227,0.029789,0.999988,0.521661,0.615691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308639,2019-09-17,9958,世紀鋼,0.128647,0.127860,0.127962,0.127603,0.192166,339531.271421,161.8,...,0.277583,0.579904,0.158289,-4.240093,0.235823,1.761640,0.070674,0.830230,0.460226,0.526650
308640,2019-09-18,9958,世紀鋼,0.128544,0.127508,0.127853,0.127215,0.188796,235454.760807,162.6,...,0.324386,0.569968,0.151184,-9.085778,0.234296,1.616571,0.068778,0.811675,0.378600,0.438244
308641,2019-09-19,9958,世紀鋼,0.128600,0.127391,0.127919,0.127087,0.187163,185428.207154,159.7,...,0.324200,0.581221,0.149689,-2.433332,0.234439,1.509105,0.066644,0.811675,0.316344,0.356600
308642,2019-09-20,9958,世紀鋼,0.128771,0.127475,0.128117,0.127184,0.186723,176377.424777,158.6,...,0.546251,0.574130,0.152988,1.722372,0.235292,1.444993,0.064494,0.815948,0.250401,0.276120


In [8]:
df_list = [group[1] for group in train_df.groupby(train_df['StockNo'])]

X_train, Y_train = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_elimination, lookback=20, forward=5, feature_list=feature_list), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_train.append(xx.tolist())
            Y_train.extend(x[1])
            

len(X_train)

191519

In [9]:
df_list = [group[1] for group in val_df.groupby(val_df['StockNo'])]

X_val, Y_val = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_elimination, lookback=20, forward=5, feature_list=feature_list), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_val.append(xx.tolist())
            Y_val.extend(x[1])

np.array(X_val).shape

(44070, 20, 48)

In [10]:
df_list = [group[1] for group in test_df.groupby(test_df['StockNo'])]

X_test, Y_test = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset, lookback=20, forward=5, feature_list=feature_list, problem='classification'), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_test.append(xx.tolist())
            Y_test.extend(x[1])

np.array(X_test).shape

(60508, 20, 48)

In [13]:
(len(X_train) == len(Y_train)) and (len(X_val) == len(Y_val)) and (len(X_test) == len(Y_test))

True

In [14]:
save_list = [X_train, Y_train, X_val, Y_val, X_test, Y_test]

In [15]:
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Cluster_{cluster}_classification_minmax0_Weekly', 'wb') as fp:
    pickle.dump(save_list, fp)

In [16]:
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Cluster_{cluster}_classification_minmax0_Weekly', 'rb') as fp:
    load_list = pickle.load(fp)

In [17]:
train_data = func.transform_cluster_csv(load_list[0], load_list[1], cluster, feature_list)
train_data

,open_day1,high_day1,low_day1,close_day1,vol_day1,VWAP_day1,index_open_day1,index_high_day1,index_low_day1,index_close_day1,...,dealer_buy_kurtosis_day20,pvt_day20,ATR_day20,RSI_15_day20,SO5_day20,SO3_day20,Y,ts,StockNo,cluster
0,0.090821,0.089773,0.091116,0.090691,0.180737,0.082629,0.623878,0.615535,0.626467,0.620752,...,0.997638,0.235328,0.048921,0.931271,0.384663,0.456011,1,2013-03-07,1338,4
1,0.091154,0.090112,0.091350,0.091080,0.180721,0.082824,0.616233,0.610531,0.615687,0.614175,...,0.997638,0.235284,0.048638,0.828305,0.367711,0.396375,1,2013-03-08,1338,4
2,0.091563,0.090524,0.091668,0.091562,0.180735,0.083087,0.617768,0.611614,0.618414,0.615267,...,0.624311,0.235265,0.047777,0.693385,0.346308,0.353686,1,2013-03-11,1338,4
3,0.092052,0.091012,0.092076,0.092140,0.180741,0.083423,0.622957,0.614320,0.626248,0.620645,...,0.339616,0.235276,0.046598,0.607681,0.329275,0.322097,1,2013-03-12,1338,4
4,0.092584,0.091546,0.092482,0.092780,0.180838,0.083756,0.629122,0.625408,0.632539,0.632984,...,0.310376,0.235294,0.045464,0.554348,0.333173,0.308648,0,2013-03-13,1338,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191514,0.350084,0.347711,0.348595,0.349449,0.182034,0.243467,0.747665,0.741986,0.748528,0.747213,...,0.560461,0.235898,0.174496,0.364985,0.624155,0.475961,0,2015-12-24,9921,4
191515,0.346814,0.344668,0.345363,0.346657,0.180943,0.241231,0.729978,0.730753,0.732635,0.732824,...,0.641649,0.236044,0.179670,0.399484,0.677869,0.592879,1,2015-12-25,9921,4
191516,0.344323,0.342305,0.342847,0.344402,0.181056,0.239499,0.725266,0.724731,0.726187,0.723509,...,0.641649,0.235413,0.175743,0.389993,0.618384,0.615563,1,2015-12-28,9921,4
191517,0.342878,0.340870,0.341266,0.342857,0.181156,0.238439,0.726613,0.719111,0.721836,0.717265,...,0.432404,0.235334,0.170788,0.420533,0.671856,0.659064,1,2015-12-29,9921,4


In [18]:
train_data.to_csv(f'D:\\OneDrive - SinoPac\\cluster_training\\cluster{cluster}_train.csv', index=False)

In [19]:
val_data = func.transform_cluster_csv(load_list[2], load_list[3], cluster, feature_list)
val_data

,open_day1,high_day1,low_day1,close_day1,vol_day1,VWAP_day1,index_open_day1,index_high_day1,index_low_day1,index_close_day1,...,dealer_buy_kurtosis_day20,pvt_day20,ATR_day20,RSI_15_day20,SO5_day20,SO3_day20,Y,ts,StockNo,cluster
0,0.293571,0.291034,0.289199,0.289174,0.184248,0.237930,0.703476,0.695406,0.693244,0.688634,...,0.173203,0.236961,0.195224,0.723259,0.663967,0.659589,0,2016-02-15,1477,4
1,0.287584,0.285535,0.283727,0.284445,0.184308,0.233381,0.672909,0.672844,0.675890,0.671545,...,0.148039,0.236731,0.194731,0.752560,0.669887,0.679893,1,2016-02-16,1477,4
2,0.281321,0.279923,0.277927,0.279707,0.183915,0.228711,0.660587,0.658213,0.656189,0.657011,...,0.138224,0.236037,0.194139,0.763188,0.639963,0.672048,1,2016-02-17,1477,4
3,0.273654,0.273151,0.271079,0.274243,0.182717,0.223141,0.632509,0.632415,0.621597,0.622112,...,0.139649,0.235562,0.193349,0.769430,0.599636,0.643873,0,2016-02-18,1477,4
4,0.270310,0.270145,0.266548,0.270428,0.184410,0.220231,0.619830,0.619707,0.621802,0.625351,...,0.163743,0.235284,0.192224,0.763985,0.512102,0.577143,1,2016-02-19,1477,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44065,0.255610,0.253327,0.253858,0.253867,0.181478,0.172834,1.086585,1.081914,1.091562,1.087331,...,0.382066,0.235523,0.124792,0.105921,0.592586,0.539320,0,2017-08-24,9921,4
44066,0.254348,0.252139,0.252604,0.252660,0.182139,0.172028,1.083884,1.080899,1.087561,1.087750,...,0.382066,0.235560,0.123643,0.175942,0.625799,0.593243,1,2017-08-25,9921,4
44067,0.253191,0.250983,0.251519,0.251520,0.182023,0.171310,1.087266,1.082842,1.094178,1.088808,...,0.397840,0.235548,0.122208,0.242671,0.655159,0.637862,1,2017-08-28,9921,4
44068,0.252084,0.249836,0.250520,0.250401,0.181439,0.170634,1.082073,1.080458,1.086933,1.079604,...,0.395344,0.235597,0.120550,0.320470,0.673303,0.671084,1,2017-08-29,9921,4


In [20]:
val_data.to_csv(f'D:\\OneDrive - SinoPac\\cluster_training\\cluster{cluster}_val.csv', index=False)

In [21]:
test_data = func.transform_cluster_csv(load_list[4], load_list[5], cluster, feature_list)
test_data

,open_day1,high_day1,low_day1,close_day1,vol_day1,VWAP_day1,index_open_day1,index_high_day1,index_low_day1,index_close_day1,...,dealer_buy_kurtosis_day20,pvt_day20,ATR_day20,RSI_15_day20,SO5_day20,SO3_day20,Y,ts,StockNo,cluster
0,0.018639,0.018791,0.018654,0.018625,0.429708,0.014913,1.106577,1.103984,1.110056,1.107944,...,0.144589,0.231479,0.010921,0.384602,0.284377,0.228374,0,2017-10-05,1314,4
1,0.018763,0.018886,0.018771,0.018738,0.486546,0.014986,1.107471,1.104520,1.109624,1.103319,...,0.144227,0.232272,0.010578,0.384602,0.310171,0.247953,0,2017-10-06,1314,4
2,0.018895,0.018987,0.018896,0.018857,0.448078,0.015062,1.108635,1.106467,1.113203,1.113050,...,0.155882,0.238220,0.010139,0.384602,0.345065,0.278727,0,2017-10-11,1314,4
3,0.019035,0.019092,0.019028,0.018981,0.308703,0.015142,1.110058,1.106844,1.111468,1.107807,...,0.139560,0.237506,0.009659,0.384602,0.372425,0.310569,0,2017-10-12,1314,4
4,0.019216,0.019230,0.019198,0.019144,0.786839,0.015247,1.108641,1.106846,1.112290,1.110408,...,0.139400,0.236100,0.009187,0.380116,0.398630,0.342249,0,2017-10-13,1314,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60503,0.122999,0.122046,0.121844,0.121737,0.184847,0.077862,1.095038,1.088501,1.092042,1.085713,...,0.350552,0.241313,0.074227,0.789602,0.766022,0.796240,1,2019-09-16,9958,4
60504,0.123140,0.122200,0.122160,0.122008,0.184480,0.078041,1.090451,1.085170,1.089305,1.083581,...,0.312014,0.240832,0.074281,0.807274,0.756965,0.800586,1,2019-09-17,9958,4
60505,0.123576,0.122675,0.122801,0.122608,0.184038,0.078432,1.068521,1.071070,1.071086,1.069821,...,0.312334,0.238208,0.074137,0.818501,0.730784,0.787014,1,2019-09-18,9958,4
60506,0.123474,0.122608,0.122830,0.122626,0.184272,0.078446,1.063745,1.068111,1.067310,1.068063,...,0.223767,0.236333,0.073782,0.818501,0.656455,0.735527,0,2019-09-19,9958,4


In [22]:
test_data.to_csv(f'D:\\OneDrive - SinoPac\\cluster_training\\cluster{cluster}_testing.csv', index=False)

In [23]:
set(val_data['StockNo'].unique().tolist()) - set(train_data['StockNo'].unique().tolist())

{'4552', '4943', '8464'}

In [24]:
set(train_data['StockNo'].unique().tolist()) - set(val_data['StockNo'].unique().tolist())

set()